In [64]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV

### Defino X,y:

In [51]:
train2=pd.read_csv("../data/train2.csv")

In [52]:
train2.sample()

,Unnamed: 0,id,carat,depth,table,price,cut_num,color_num,clarity_num
15307,15307,15307,0.5,59.3,59.0,1410,5,6,6


In [53]:
X=train2.drop(columns=["id","price","Unnamed: 0"])

In [54]:
y=train2.price

### Train, test, split:

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [57]:
X_train.shape

(32364, 6)

In [58]:
X_test.shape

(8091, 6)

In [59]:
y_train.shape

(32364,)

In [60]:
y_test.shape

(8091,)

### Entrenamos varios Modelos de Regresión (variables cuantitativas).

In [86]:
models={'lin':LinearRegression(),
    'ridge': Ridge(),
    'lasso': Lasso(),
    'sgd': SGDRegressor(),
    'knn': KNeighborsRegressor(),
    'grad': GradientBoostingRegressor(),
    'elas': ElasticNet(),
    'svr' :LinearSVR(),
    'tree': DecisionTreeRegressor(),
    'random': RandomForestRegressor()
        }

In [166]:
lista_modelos=[]
for name, model in models.items():
    lista_modelos.append(model)
   

In [167]:
lista_modelos

[LinearRegression(),
 Ridge(),
 Lasso(),
 SGDRegressor(),
 KNeighborsRegressor(),
 GradientBoostingRegressor(),
 ElasticNet(),
 LinearSVR(),
 DecisionTreeRegressor(),
 RandomForestRegressor()]

In [131]:
parameters_lin={'copy_X': [True,False],
                'fit_intercept': [True,False],
                'n_jobs': [-2,-1,1,2,None],  
                'positive': [True,False]
               }

In [133]:
parameters_ridge={'alpha': [1.0,2.0,3.0,4.0], 
                 'copy_X': [True,False], 
                 'fit_intercept': [True,False], 
                 'max_iter': [1000, 15000, None],  
                 'positive': [True,False], 
                 'random_state':[1,2,3,4,5,None] , 
                 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg'], 
                 'tol': [0.001,0.002,0.003]
                 }

In [134]:
parameters_lasso={'alpha': [0.0,1.0], 
                  'copy_X': [True,False], 
                  'fit_intercept': [True,False], 
                  'max_iter': [500,1000,1500],  
                  'positive': [True,False], 
                  'precompute': [True,False], 
                  'random_state': [True,False], 
                  'selection': ['cyclic','random'], 
                  'tol': [0.0001, 0.0002],
                  'warm_start': [True,False]
                 }

In [137]:
parameters_sgd={'alpha': [0.0001,0.001,0.01], 
                'average': [True,False], 
                'early_stopping': [True,False], 
                'epsilon': [0.1,0.2,0.3], 
                'eta0': [0.1,0.2,0.3], 
                'fit_intercept': [True,False], 
                'l1_ratio': [0,0.15,0.5,0.75,1.0], 
                'learning_rate': ['invscaling','adaptive','constant','constant'], 
                'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                'max_iter': [500,1000,2000], 
                'n_iter_no_change': [2,5,10], 
                'penalty': ['l2','l1', 'elasticnet'], 
                'power_t': [0.25, 0.5],  
                'shuffle': [True,False], 
                'tol': [0.001, None], 
                'validation_fraction': [0.0,0.1,0.5,0.75,1.0], 
                'verbose': [0,1,2], 
                'warm_start': [True,False]
               }

In [138]:
parameters_knn={'algorithm': ['auto','ball_tree', 'kd_tree', 'brute'], 
                'leaf_size': [20,30,40], 
                'metric': ['minkowski',"precomputed"],  
                'n_jobs': [-1,None], 
                'n_neighbors':[1,2,3,4,5,6,7,8,9], 
                'p': [1,2], 
                'weights': ['uniform','distance']
               }

In [140]:
parameters_grad={'alpha': [0.1, 0.5, 0.9], 
                 'ccp_alpha': [0.0,0.1, 0.5, 0.9], 
                 'criterion': ['friedman_mse', 'squared_error', 'mse', 'mae'],  
                 'learning_rate': [0.1, 0.5], 
                 'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'], 
                 'max_depth': [1,2,3,4,5], 
                 'max_features': ['auto', 'sqrt', 'log2',None], 
                 'max_leaf_nodes':[50,100,200, None], 
                 'min_impurity_decrease': [0.0,0.1,0.5,1.0], 
                 'min_samples_leaf': [1,2,3,4,5], 
                 'min_samples_split': [1,2,3,4,5], 
                 'min_weight_fraction_leaf': [0.0, 0.5],
                 'n_estimators': [100, 200,300, 500], 
                 'n_iter_no_change': [2,5,10,None],  
                 'subsample': [0.1,0.5,1.0], 
                 'tol': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
                 'validation_fraction': [0.0,0.1,0.5, 0.75,1.0], 
                 'verbose': [0,1,2],
                 'warm_start': [True,False]}

In [141]:
parameter_elast = {'alpha': [1.0, 2.0, 3.0, 4.0, 5.0], 
                   'copy_X': [True, False],
                   'fit_intercept': [True, False],
                   'l1_ratio': [0.1, 0.5, 1.0], 
                   'max_iter': [1000, 2000, 3000, 4000, 5000], 
                   'positive': [True, False],
                   'precompute': [True, False],
                   'selection': ['cyclic', 'random'], 
                   'tol': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
                   'warm_start': [True, False],
                  }

In [142]:
parameters_svr = {'C': [1.0, 2.0, 3.0, 4.0, 5.0], 
                  'dual': [True, False], 
                  'epsilon': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0],
                  'fit_intercept': [True, False], 
                  'intercept_scaling': [1.0, 2.0, 3.0, 4.0, 5.0],
                  'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'], 
                  'max_iter': [1000, 2000, 3000, 4000, 5000], 
                  'tol': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
                  'verbose': [0, 1, 2]
                 }

In [143]:
paremeters_tree = {'ccp_alpha': [0.0, 0.1, 0.2],
                   'criterion': 'squared_error', 
                   'max_depth': [0,1,2,3,4,5,None], 
                   'max_features': ["auto", "sqrt", "log2", None],
                   'max_leaf_nodes': [0,1,2,3,4,5,None], 
                   'min_impurity_decrease': [0.0, 0.1, 0.2],
                   'min_samples_leaf': [1,2,3,4,5], 
                   'min_samples_split': [1,2,3,4,5],
                   'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
                   'splitter': ['best', 'random']
                  }

In [129]:
parameters_rand = {'bootstrap': [True, False],
                   'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                   'max_features': ['auto', 'sqrt'],
                   'min_samples_leaf': [1, 2, 4],
                    'min_samples_split': [2, 5, 10],
                    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
                  }

In [146]:
dicc_parametros={"parameters_lin":parameters_lin,"parameters_ridge":parameters_ridge,"parameters_lasso":parameters_lasso,"parameters_sgd": parameters_sgd,"parameters_knn":parameters_knn,"parameters_grad":parameters_grad,"parameter_elast":parameter_elast,"parameters_svr":parameters_svr,"paremeters_tree":paremeters_tree,"parameters_rand":parameters_rand}

In [151]:
grid = GridSearchCV(lin, parameters_lin, verbose=1)

In [152]:
print(grid)

GridSearchCV(estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'n_jobs': [-2, -1, 1, 2, None],
                         'positive': [True, False]},
             verbose=1)


In [153]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'n_jobs': [-2, -1, 1, 2, None],
                         'positive': [True, False]},
             verbose=1)

In [155]:
print(grid.best_params_)

{'copy_X': True, 'fit_intercept': True, 'n_jobs': -2, 'positive': False}


In [108]:
for name, model in models.items():
    print("ENTRENANDO: ", name)
    model.fit(X_train, y_train)


ENTRENANDO:  lin
ENTRENANDO:  ridge
ENTRENANDO:  lasso
ENTRENANDO:  sgd
ENTRENANDO:  knn
ENTRENANDO:  grad
ENTRENANDO:  elas
ENTRENANDO:  svr
ENTRENANDO:  tree
ENTRENANDO:  random


In [100]:
lin.get_params(deep=True)

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': 'deprecated',
 'positive': False}

In [89]:
print(parametros)

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': 'deprecated', 'positive': False}


In [ ]:
for elemento in lista:
    grid = GridSearchCV(elemento, params, verbose=1)

In [46]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"------{name}------")
    print('RMSE - ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

------lin------
RMSE -  1194.5209264818127
------ridge------
RMSE -  1194.5247227816853
------lasso------
RMSE -  1194.5168622802832
------sgd------
RMSE -  276344864.6974937
------knn------
RMSE -  1907.5574307333873
------grad------
RMSE -  602.309383130032
------elas------
RMSE -  2915.606909356901
------svr------
RMSE -  1859.0781715959422
